# LA County Board of Supervisors
An analysis of donations to 2nd District candidates

### Read libraries

In [41]:
import json
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
from matplotlib import rc
%matplotlib inline
import numpy as np
import geojson
import jenkspy
import altair as alt
alt.renderers.enable('notebook')
import altair as alt
import altair_latimes as lat
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

### Load full list of donations to county candidates since 2007

In [42]:
contributions = pd.read_csv('input/contributions.csv',low_memory=False, dtype={'contributor_zip_code': 'str', 'contributor_zip_code_ext': 'str'})

contributions.contributor_address_2 = contributions.contributor_address_2.fillna('')
contributions.occupation = contributions.occupation.fillna('')
contributions.employer = contributions.employer.fillna('')
contributions.description = contributions.description.fillna('')
contributions.contributor_first_name = contributions.contributor_first_name.fillna('')
contributions.contributor_zip_code_ext = contributions.contributor_zip_code_ext.fillna('')

In [43]:
contributions.columns

Index(['last_name', 'first_name', 'committee_id', 'committee_name',
       'office_type', 'district_number', 'schedule', 'type', 'period_beg_date',
       'period_end_date', 'election_date', 'date', 'amount_rcvd', 'amount_pd',
       'description', 'contributor_first_name', 'contributor_last_name',
       'contributor_address', 'contributor_address_2', 'contributor_city',
       'contributor_state', 'contributor_zip_code', 'contributor_zip_code_ext',
       'occupation', 'employer', 'rec_id', 'cycle'],
      dtype='object')

### Clean up the table

In [44]:
#names
contributions['amount_rcvd'] = contributions['amount_rcvd'].astype(int)
contributions['monthyear'] = pd.to_datetime(contributions['date']).map(lambda dt: dt.replace(day=1))
contributions['type'] = contributions['type'].str.replace("Small Contributor Committee", "Small Donor Committee")
contributions['first_name'] = contributions['first_name'].str.upper()
contributions['last_name'] = contributions['last_name'].str.upper()
contributions['occupation'] = contributions['occupation'].str.upper()
contributions['employer'] = contributions['employer'].str.upper()
contributions['committee_name'] = contributions['committee_name'].str.upper()
contributions['candidate_name'] =  contributions["first_name"].str.title() + ' ' + contributions["last_name"].str.title()
contributions['candidate_name'] = contributions['candidate_name'].str.replace('Holly J. Mitchell', 'Holly Mitchell')
contributions['first_name'] = contributions['first_name'].str.replace('HOLLY J.', 'HOLLY')
contributions["nu_contributor_address"] = contributions["contributor_address"].str.split('#').str[0].str.split(', ').str[0].str.split('Ste').str[0].str.split('Suite').str[0].str.split(',').str[0]
contributions['full_address'] =  contributions["nu_contributor_address"] + ', ' + contributions["contributor_city"] + ', ' + contributions["contributor_state"] + ', ' + contributions["contributor_zip_code"].astype(str)
contributions['candidate_slug'] =  contributions["first_name"].str.lower() + '_' + contributions["last_name"].str.lower()
contributions['candidate_slug'] = contributions['candidate_slug'].str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace('.', '').str.replace('\'', '')

#time
contributions['date'] = pd.to_datetime(contributions['date'])
contributions['year'], contributions['month'] = contributions['date'].dt.year, contributions['date'].dt.month
contributions['year'] = contributions['year'].astype(str)
contributions['month'] = contributions['month'].astype(str)
contributions['months'] = contributions['date'].dt.strftime('%m/%Y').astype(str)
contributions['week'] = contributions['date'].dt.week
contributions['week'] = contributions['week'].astype(str)

### Filter data for recent contributions and money in the District 2 race

In [32]:
recent_contributions = contributions[(contributions['date'] > '2018-06-30')]
second_district_candidates = (['Holly Mitchell','Herb Wesson','Jan Perry','Chan Jeong'])
districttwo = recent_contributions[recent_contributions['candidate_name'].isin(second_district_candidates)]

In [33]:
districttwo[['candidate_name','contributor_first_name', 'contributor_last_name','occupation', 'employer','amount_rcvd']]

In [34]:
districttwo_indiv = districttwo[districttwo.type == 'I']

In [35]:
districttwo_indiv

### How much did each occupation give?

In [36]:
second_district_occupations = districttwo_indiv.groupby(['occupation'])['amount_rcvd'].sum().reset_index().sort_values(by='occupation', ascending=True)

In [37]:
# second_candidates_occupations_pivot = pd.pivot_table(second_district_occupations,values='amount_rcvd', index=['occupation'], columns=['candidate_name'], aggfunc=[np.sum], fill_value='0').reset_index()

In [38]:
second_district_occupations

In [39]:
contributions.columns

### Specific donor lookup

In [40]:
contributions[contributions['contributor_last_name'].str.contains('Juul')]